# **Assignment 7 : Text Classification**
### **Krishna Savaliya(202011050)**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#unzip the dataset to the content
# dataset7 : sample_submission.csv, train.csv, test.csv, test_labels.csv
from zipfile import ZipFile
with ZipFile('/content/drive/MyDrive/dataset7.zip', 'r') as zipObj:
   # Extract all the contents of zip file in different directory
   zipObj.extractall('/content/')

In [5]:
#Imported required libraries here
import os
import re
import math
import glob
import numpy as np
from numpy.linalg import norm
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords,wordnet
from nltk.stem import PorterStemmer,WordNetLemmatizer
import pandas as pd
from IPython.display import display
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity
import pickle
import collections
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

## Preprocessing of Data

In [16]:
class Preprocessing_Data: 
  def load_data(self, type_of_data):      #To Load the Data Available for applying preprocessing on it
    global total
    total = 0     
    data = list()         #contains list of list of data for each file. each list within list has one corresponding file data
    if type_of_data == "train":
      location = "/content/dataset7/train.csv"
    else:
      location = "/content/dataset7/test.csv"        
    df = pd.read_csv(location)    #reading csv file 
    df['comment_text'].replace('', np.nan, inplace=True) 
    df.dropna(subset=['comment_text'], inplace=True)
    if type_of_data == "train":
      pickle.dump(df, open('/content/drive/MyDrive/IR/dump/train_data','wb'))
    else:
      pickle.dump(df, open('/content/drive/MyDrive/IR/dump/test_data/test_data_main','wb'))
      test_labels = pd.read_csv("/content/dataset7/test_labels.csv")
      test_labels = pd.merge(test_labels, df, on = "id", how = "right")
      pickle.dump(test_labels, open('/content/drive/MyDrive/IR/dump/test_data/test_labels_data','wb'))
      print(test_labels.shape)
      
    comments = list(df.loc[:,"comment_text"])
    comment_ids = list(df.loc[:,"id"])
    new_comment_ids = list()
    for comment_id,comment in zip(comment_ids,comments):                               
      data.append(comment)       
      new_comment_ids.append(comment_id)
      total += 1
    return new_comment_ids,data

  #function to preprocess the data
  def preprocess(self,data,type_of_data):
      #retrieving each doc data and then removing numerical and punctuation from each document
      stop_words = stopwords.words('english')
      lemmatizer = WordNetLemmatizer()
      preprocessed_data = list()
      if type_of_data == "train":
        xdata = pickle.load(open('/content/drive/MyDrive/IR/dump/train_data','rb'))
      else:
        xdata = pickle.load(open('/content/drive/MyDrive/IR/dump/test_data/test_data_main','rb'))
        test_labels = pickle.load(open('/content/drive/MyDrive/IR/dump/test_data/test_labels_data','rb'))
      drop_index = list()
      for index,doc_text in zip(xdata.index, data):
        #remooving numerical and punctuation
        doc_text = re.sub(r'[^\w\s]', '',doc_text)
        doc_text = re.sub('\d','',doc_text)
        #data tokenizing
        data_tokenized = nltk.word_tokenize(doc_text)
        #converting to smallcase 
        data_smallcase = [word.lower() for word in data_tokenized] 
        #removing stopwords
        data_without_stopwords = list()       
        for word in data_smallcase:
          if word not in stop_words:
            data_without_stopwords.append(word)
        #contractions
        data_contractions = map(expand_contractions, data_without_stopwords)
        #Lemmatization
        prev_data = list() 
        for word in data_contractions:
          prev_data.append(lemmatizer.lemmatize(word,pos='v'))
        if type_of_data == "train":
          test_row = ["dummy"]
          test_type = ["should not be equal"] 
        else:
          test_row = list(test_labels.loc[index,["toxic","severe_toxic","obscene","threat","insult","identity_hate"]])
          test_type= [-1]*6
          
        if len(prev_data) == 0 or (test_row == test_type):
          drop_index.append(index)
        else:
          preprocessed_data.append(prev_data)
         
      df = xdata.drop(drop_index)
      if type_of_data == "train":
        pickle.dump(df,open("/content/drive/MyDrive/IR/dump/train_data","wb"))
      else:
        test_labels = pd.merge(test_labels,df, on = "id", how = "right")
        pickle.dump(test_labels, open('/content/drive/MyDrive/IR/dump/test_data/test_labels_data','wb'))
        print(test_labels.shape)
        pickle.dump(df,open("/content/drive/MyDrive/IR/dump/test_data/test_data_main","wb"))
        
      return preprocessed_data
  
  def extractUnique(self,data):
    #return unique feature from the entire corpus
    return collections.Counter([x for sub_list in data for x in sub_list])

In [11]:
#creating preprocessing object class
p1 = Preprocessing_Data()
#loading the data
comment_ids,data = p1.load_data("train")
#preprocessing data
preprocessed_data = p1.preprocess(data,"train")

In [12]:
#As the data has been preprocessed dumping it so that we do not require to run the preprocessing step again 
pickle.dump(preprocessed_data, open('/content/drive/MyDrive/IR/dump/comment_preprocessed','wb'))
pickle.dump(comment_ids,open('/content/drive/MyDrive/IR/dump/comment_ids','wb'))
infile = open('/content/drive/MyDrive/IR/dump/comment_preprocessed','rb')
preprocessed_data = pickle.load(infile)
infile.close()
infile = open('/content/drive/MyDrive/IR/dump/comment_ids','rb')
doc = pickle.load(infile)
infile.close()
total = len(preprocessed_data)
print("Number of documents=",total)
feature_collection = p1.extractUnique(preprocessed_data)
feature_list= list(feature_collection)
comment_feature_list = list()
for document in preprocessed_data:
  c = collections.Counter([word for word in document])
  comment_feature_list.append(c)

Number of documents= 159525


## TF_IDF Representation

TFIDF class:

In [13]:
class TF_IDF:  
  def __init__(self):   
    self.idf_matrix = csr_matrix(self.find_idf_matrix1())
    
  #function to fit the data
  def fit(self,comments):
    global feature_list
    #indices for generating sparse matrix 
    global comment_feature_list
    rows = list()
    columns = list() 
    values = list() 
  
    reverse_feature_list = dict()
    for i,word in enumerate(feature_list):
      reverse_feature_list[word] = i 
    
    for i,document in zip(range(len(comments)),feature_list):
      total_words_in_doc = len(document)
      #if i%10000 == 0 :
        #print("document processed=",i)
      
      for word in comment_feature_list[i]:
        count = comment_feature_list[i][word]
        j = reverse_feature_list.get(word,-1)
        if j == -1:
          continue
        rows.append(i)
        columns.append(j)
        values.append(float(count)/total_words_in_doc)       #term frequency
    tf_sparse = csr_matrix((values, (rows, columns)), shape=(len(comments), len(feature_list)))
    return csr_matrix.multiply(tf_sparse, self.idf_matrix)       #tf*idf 

  #function to transform the query to tf-idf representation
  def transform(self, test_comments):
    global feature_list 
    global test_comment_feature_list

    #indices for generating sparse matrix  
    rows = list()
    columns = list() 
    values = list() 

    reverse_feature_list = dict()
    for i,word in enumerate(feature_list):
      reverse_feature_list[word] = i
    for i, document in zip(range(len(test_comments)),feature_list):
      total_words_in_doc = len(document)
      for word in test_comment_feature_list[i]:
          count = test_comment_feature_list[i][word]
          j = reverse_feature_list.get(word,-1)
          if j == -1:
            continue
          element_value = float(count)/total_words_in_doc
          rows.append(i)
          columns.append(j)
          values.append(element_value) 
    tf_sparse = csr_matrix((values, (rows, columns)), shape=(len(test_comments), len(feature_list)))
    return csr_matrix.multiply(tf_sparse,self.idf_matrix)

  def return_feature_generator(self):
    global feature_list 
    for ele in feature_list: 
      yield (ele)
      
  def find_idf_matrix1(self):
    #return list which stores the idf value of each unique term
    global feature_list
    global feature_collection 
    global preprocessed_data 
    temp_matrix = np.zeros((1,len(feature_list)))
    feature_list_generator = self.return_feature_generator()
    for i,ele in zip(range(len(feature_list)),feature_list_generator):
      temp_matrix[0,i] = feature_collection[ele]
    return np.log10(np.multiply(total,np.reciprocal(temp_matrix)))

TFIDF fit

In [14]:
#generating object for preprocesssing object as previous generator has already been used 
tfidf_obj = TF_IDF()
term_doc_matrix = tfidf_obj.fit(preprocessed_data)        #passing preprocessed generator
print(term_doc_matrix.shape)
#generated object for term_doc_matrix generator cannot print shape because the class is generator
del feature_collection 
del comment_feature_list

(159525, 215397)


TFIDF transform comments

In [18]:
#loading test_comments 
p2 = Preprocessing_Data()
test_comment_ids , test_comments = p2.load_data("test")

#Preprocessing the test data
test_comment_generator = p2.preprocess(test_comments,"test")
test_comment_feature_list = list()
for comment in test_comment_generator:
  c = collections.Counter([word for word in comment])
  test_comment_feature_list.append(c)

#transforming test_comments
transformed_comments = tfidf_obj.transform(test_comment_generator)                           #will contain all the transformed test comments

pickle.dump(transformed_comments, open('/content/drive/MyDrive/IR/dump/transformed_comments','wb'))

(153164, 8)
(63929, 9)


Making function to calculate prision, recall and fscore 

In [19]:
import sklearn
#making a utility function that calculates precision, recall and fscore
def fscore(average, y_test, y_pred, labels):
  return sklearn.metrics.precision_recall_fscore_support(y_test, y_pred, beta=1.0,labels=[labels]
                                                ,average=average, warn_for={'precision', 'recall', 'f-score'}
                                                ,zero_division='warn')

Using SVM for classification

In [22]:
from sklearn.svm import LinearSVC
from sklearn.multioutput import MultiOutputClassifier
  
# Building a Support Vector Machine on train data so here we are using svm classifier.  
svc = LinearSVC()
x_train = term_doc_matrix
df = pickle.load(open("/content/drive/MyDrive/IR/dump/train_data","rb"))
y_train  = np.array(df.loc[:,["toxic","severe_toxic","obscene","threat","insult","identity_hate"]])
svc_multioutput_classifier = MultiOutputClassifier(svc,n_jobs = -1)
svc_multioutput_classifier = svc_multioutput_classifier.fit(x_train, y_train)

#save the classifier data 
pickle.dump(svc_multioutput_classifier,open("/content/drive/MyDrive/IR/dump/svc_multioutput","wb"))

column_labels = ["toxic","severe_toxic","obscene","threat","insult","identity_hate"]
transformed_comments = pickle.load(open("/content/drive/MyDrive/IR/dump/transformed_comments","rb"))
svc_multioutput_classifier = pickle.load(open("/content/drive/MyDrive/IR/dump/svc_multioutput","rb"))
y_test = pickle.load(open("/content/drive/MyDrive/IR/dump/test_data/test_labels_data","rb"))
y_test = np.array(y_test.loc[:,column_labels])

#SVM prediction and result of classification
y_pred= svc_multioutput_classifier.predict(transformed_comments)
for i in range(len(column_labels)):
  print("\nClass : ",column_labels[i])
  print("\n\nMicro : \n")
  micro_scores = fscore("micro",y_test,y_pred,i)
  print("Precision score : ",micro_scores[0])
  print("Recall score : ",micro_scores[1])
  print("F-score score : ",micro_scores[2])
  print("\n")
  print("\nMacro : \n")
  macro_scores = fscore("macro",y_test,y_pred,i)
  print("Precision score : ",macro_scores[0])
  print("Recall score : ",macro_scores[1])
  print("F-score score : ",macro_scores[2])
  print("\n------------------------------------------")


Class :  toxic


Micro :

Precision score :  0.5493636475966885
Recall score :  0.7300492610837438
F-score score :  0.6269477543538038



Macro :

Precision score :  0.5493636475966885
Recall score :  0.7300492610837438
F-score score :  0.6269477543538038

------------------------------------------

Class :  severe_toxic


Micro :

Precision score :  0.3271889400921659
Recall score :  0.3869209809264305
F-score score :  0.3545568039950063



Macro :

Precision score :  0.3271889400921659
Recall score :  0.3869209809264305
F-score score :  0.3545568039950063

------------------------------------------

Class :  obscene


Micro :

Precision score :  0.6127314227745372
Recall score :  0.6545651584936332
F-score score :  0.632957820277705



Macro :

Precision score :  0.6127314227745372
Recall score :  0.6545651584936332
F-score score :  0.632957820277705

------------------------------------------

Class :  threat


Micro :

Precision score :  0.21818181818181817
Recall score :  0.22748

## Word2Vec Representation

In [23]:
comment_preprocessed = pickle.load(open("/content/drive/MyDrive/IR/dump/comment_preprocessed","rb"))
from gensim.models import Word2Vec
skipgram_model = Word2Vec(comment_preprocessed, size=300, window=10, min_count=1, sg=1, workers=4, alpha=0.025, iter=5, negative=0)  #skipgram model of window size = 10 and size = 300

In [24]:
#saving model
pickle.dump(skipgram_model,open("/content/drive/MyDrive/IR/dump/skipgram_model","wb"))

Vectorizing training comments:

In [25]:
train_comments = np.zeros((len(comment_preprocessed),300))
count = 0
for i in comment_preprocessed:
  temp = np.zeros((300))
  for j in i:
    try:
      temp = temp + skipgram_model[j]
    except:
      continue
  train_comments[count,:] = temp
  count = count + 1 
print("Length of train array:",len(train_comments))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


Length of train array: 159525


Vectorizing test comments:

In [26]:
test_comments = np.zeros((len(test_comment_generator),300)) 
count = 0
for i in test_comment_generator:
  temp = np.zeros((300))
  for j in i:
      try:
        temp = temp + skipgram_model[j]
      except:
        continue
  test_comments[count,:] = temp
  count = count + 1
print("Length of test array:",len(test_comments))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


Length of test array: 63929


Using SVM for classification

In [28]:
from sklearn.svm import LinearSVC
from sklearn.multioutput import MultiOutputClassifier
  
# Building a Support Vector Machine on train data so here we are using svm classifier.  
svc = LinearSVC()
x_train = train_comments
df = pickle.load(open("/content/drive/MyDrive/IR/dump/train_data","rb"))
y_train  = np.array(df.loc[:,["toxic","severe_toxic","obscene","threat","insult","identity_hate"]])
svc_multioutput_classifier = MultiOutputClassifier(svc,n_jobs = -1)
svc_multioutput_classifier = svc_multioutput_classifier.fit(x_train, y_train)

#save the classifier data 
pickle.dump(svc_multioutput_classifier,open("/content/drive/MyDrive/IR/dump/svc_multioutput_word","wb"))

column_labels = ["toxic","severe_toxic","obscene","threat","insult","identity_hate"]
transformed_comments = test_comments
svc_multioutput_classifier = pickle.load(open("/content/drive/MyDrive/IR/dump/svc_multioutput_word","rb"))
y_test = pickle.load(open("/content/drive/MyDrive/IR/dump/test_data/test_labels_data","rb"))
y_test = np.array(y_test.loc[:,column_labels])

#SVM prediction and results of classification
y_pred= svc_multioutput_classifier.predict(transformed_comments)
for i in range(len(column_labels)):
  print("\nClass : ",column_labels[i])
  print("\n\nMicro : \n")
  micro_scores = fscore("micro",y_test,y_pred,i)
  print("Precision score : ",micro_scores[0])
  print("Recall score : ",micro_scores[1])
  print("F-score score : ",micro_scores[2])
  print("\n")
  print("\nMacro : \n")
  macro_scores = fscore("macro",y_test,y_pred,i)
  print("Precision score : ",macro_scores[0])
  print("Recall score : ",macro_scores[1])
  print("F-score score : ",macro_scores[2])
  print("\n-----------------------------------------")

/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning



Class :  toxic


Micro :

Precision score :  0.6542553191489362
Recall score :  0.08078817733990148
F-score score :  0.143817597193803



Macro :

Precision score :  0.6542553191489362
Recall score :  0.08078817733990148
F-score score :  0.143817597193803

-----------------------------------------

Class :  severe_toxic


Micro :

Precision score :  0.43373493975903615
Recall score :  0.09809264305177112
F-score score :  0.16000000000000003



Macro :

Precision score :  0.43373493975903615
Recall score :  0.09809264305177112
F-score score :  0.16000000000000003

-----------------------------------------

Class :  obscene


Micro :

Precision score :  0.7177242888402626
Recall score :  0.08886480628555947
F-score score :  0.15814850530376084



Macro :

Precision score :  0.7177242888402626
Recall score :  0.08886480628555947
F-score score :  0.15814850530376084

-----------------------------------------

Class :  threat


Micro :

Precision score :  0.22727272727272727
Recall score :

In [10]:
import re
contractions_dict = {
'didn\'t': 'did not',
'don\'t': 'do not',
"ain't": "am not / are not / is not / has not / have not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is / how does",
"I'd": "I had / I would",
"I'd've": "I would have",
"I'll": "I shall / I will",
"I'll've": "I shall have / I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}
contractions_re = re.compile('(%s)' % '|'.join(contractions_dict.keys()))
def expand_contractions(s, contractions_dict=contractions_dict):  #contraction function
    def replace(match):                                              
         return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, s)                        #replacing key with by above defined dictionary